Ensemble of the three notebooks.Please upvote the original notebooks:

**1. Deberta v3 large_transformer_head**
[Deberta_V3_Large_Transformer_Head](https://www.kaggle.com/code/jujutsuns/deberta-v3-large-transformer-head?scriptVersionId=95340789)

**2. anferico/bert-for-patents**
[patent_phrase_matching [inference]](https://www.kaggle.com/code/barangizagiza/patent-phrase-matching-inference?scriptVersionId=98350930)

**3. anferico/bert-for-patents**
[tez inference phrase matching base](https://www.kaggle.com/code/jujutsuns/tez-inference-phrase-matching-skf?scriptVersionId=97967285)

**4. RoBERTa-large**
[PatentPhrase RoBERTa Inference](https://www.kaggle.com/code/santhoshkumarv/patentphrase-roberta-inference-lb-0-814)

**5. Deberta v3 large**
[Deberta v3 large + additional fold(s)](https://www.kaggle.com/code/renokan/pppm-deberta-v3-large-additional-fold-s/notebook?scriptVersionId=96750400)

**6. Deberta v3 large_transformer_head**
[Deberta_V3_Large_Transformer_Head](https://www.kaggle.com/code/jujutsuns/deberta-v3-large-transformer-head?scriptVersionId=96208185)

**7. Electra**
[Electra](https://www.kaggle.com/code/gaozhao/a-simple-ensemble-of-two/notebook)

**8. Electra**
[Electra](https://www.kaggle.com/code/jujutsuns/electra?scriptVersionId=98672999)

**9. xlm roberta large**
[xlm roberta large [inference]](https://www.kaggle.com/code/jujutsuns/xlm-roberta-large-inference?scriptVersionId=98231498)

And use ensemble strategy from:
[Tips for ensambling](https://www.kaggle.com/code/jellyz9/tips-for-ensambling)

# 1.Deberta v3 large

In [1]:
# ====================================================
# Directory settings
# ====================================================
import os

INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [2]:
# ====================================================
# CFG
# ====================================================
class CFG:
    num_workers=4
    path="../input/pppm-deberta-v3-large-baseline-w-w-b-train/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=133
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]

In [3]:
import os
import gc
import math
import time
import random
import scipy as sp
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection
from dataclasses import dataclass
from typing import Optional
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")

from transformers import AdamW, AutoConfig, AutoModel, AutoTokenizer, get_cosine_schedule_with_warmup
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForTokenClassification, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import TrainingArguments, Trainer
%env TOKENIZERS_PARALLELISM=true


import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Found existing installation: transformers 4.18.0
Uninstalling transformers-4.18.0:
  Successfully uninstalled transformers-4.18.0


Found existing installation: tokenizers 0.12.1
Uninstalling tokenizers-0.12.1:
  Successfully uninstalled tokenizers-0.12.1


Looking in links: ../input/pppm-pip-wheels-dataset
Processing /kaggle/input/pppm-pip-wheels-dataset/transformers-4.16.2-py3-none-any.whl
Processing /kaggle/input/pppm-pip-wheels-dataset/tokenizers-0.11.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


Looking in links: ../input/pppm-pip-wheels-dataset


tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2
tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2
env: TOKENIZERS_PARALLELISM=true


In [4]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [5]:
# ====================================================
# Data Loading
# ====================================================
test = pd.read_csv(INPUT_DIR+'test.csv')

In [6]:
# ====================================================
# CPC Data
# ====================================================
cpc_texts = torch.load(CFG.path+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())

,id,anchor,target,context,context_text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,PHYSICS. OPTICS
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...
2,36baf228038e314b,lower trunnion,lower locating,B60,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...
3,1f37ead645e7f0c8,cap component,upper portion,D06,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE


In [7]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

,id,anchor,target,context,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,PHYSICS. OPTICS,opc drum[SEP]inorganic photoconductor drum[SEP...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow[SEP]altering gas flow[SEP]MECH...
2,36baf228038e314b,lower trunnion,lower locating,B60,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion[SEP]lower locating[SEP]PERFORMI...
3,1f37ead645e7f0c8,cap component,upper portion,D06,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component[SEP]upper portion[SEP]TEXTILES; ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation[SEP]artificial neural netwo...


In [8]:
# ====================================================
# tokenizer
# ====================================================
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

In [9]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

In [10]:
class TransformerHead(nn.Module):
    def __init__(self, in_features, max_length, num_layers=1, nhead=8, num_targets=1):
        super().__init__()

        self.transformer = nn.TransformerEncoder(encoder_layer=nn.TransformerEncoderLayer(d_model=in_features,
                                                                                          nhead=nhead),
                                                 num_layers=num_layers)
        self.row_fc = nn.Linear(in_features, 1)
        self.out_features = max_length

    def forward(self, x):
        out = self.transformer(x)
        out = self.row_fc(out).squeeze(-1)
        return out

In [11]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        
        self.feature_extractor = AutoModelForTokenClassification.from_pretrained("../input/deberta-v3-large/deberta-v3-large")
        in_features = self.feature_extractor.classifier.in_features
        self.attention = TransformerHead(in_features=in_features, max_length=133, num_layers=1, nhead=8, num_targets=1)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.attention.out_features, self.cfg.target_size)
        self._init_weights(self.fc)
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        feature = self.attention(last_hidden_states)
        
        return feature


    def forward(self, inputs):
        feature = self.feature(inputs)
        #print(feature.shape)
        output = self.fc(self.fc_dropout(feature))
        return output

In [12]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [13]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(f"../input/debertalargetransformerheadfold{fold}/uspppmmicrosoft-deberta-v3-large_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
pred1 = np.mean(predictions, axis=0)

Some weights of the model checkpoint at ../input/deberta-v3-large/deberta-v3-large were not used when initializing DebertaV2ForTokenClassification: ['mask_predictions.classifer.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'deberta.embeddings.position_embeddings.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifer.weight']
- This IS expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model that you expect to be exactly identical (in

In [14]:
MMscaler = MinMaxScaler()
pred1_mm = MMscaler.fit_transform(pred1.reshape(-1,1)).reshape(-1)

In [15]:
submission_1 = pd.DataFrame({
    'id': test['id'],
    'score': pred1_mm,
})

submission_1.head(14)

,id,score
0,4112d61851461f60,0.498692
1,09e418c93a776564,0.722397
2,36baf228038e314b,0.449420
3,1f37ead645e7f0c8,0.254730
4,71a5b6ad068d531f,0.002512
5,474c874d0c07bd21,0.467280
6,442c114ed5c4e3c9,0.459023
7,b8ae62ea5e1d8bdb,0.000562
8,faaddaf8fcba8a3f,0.295239
9,ae0262c02566d2ce,1.000000


In [16]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(f"../input/debertalargetransformerheadgkf{fold}/uspppmmicrosoft-deberta-v3-large_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
pred6 = np.mean(predictions, axis=0)

Some weights of the model checkpoint at ../input/deberta-v3-large/deberta-v3-large were not used when initializing DebertaV2ForTokenClassification: ['mask_predictions.classifer.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'deberta.embeddings.position_embeddings.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifer.weight']
- This IS expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model that you expect to be exactly identical (in

In [17]:
MMscaler = MinMaxScaler()
pred6_mm = MMscaler.fit_transform(pred6.reshape(-1,1)).reshape(-1)

In [18]:
submission_6 = pd.DataFrame({
    'id': test['id'],
    'score': pred6_mm,
})

submission_6.head(14)

,id,score
0,4112d61851461f60,0.511849
1,09e418c93a776564,0.734401
2,36baf228038e314b,0.479310
3,1f37ead645e7f0c8,0.272812
4,71a5b6ad068d531f,0.002929
5,474c874d0c07bd21,0.449986
6,442c114ed5c4e3c9,0.442169
7,b8ae62ea5e1d8bdb,0.000617
8,faaddaf8fcba8a3f,0.275198
9,ae0262c02566d2ce,1.000000


# 2.anferico/bert-for-patents

In [19]:
class CFG:
    fold_num = 5
    input_path = '../input/us-patent-phrase-to-phrase-matching/'
    model_path = '../input/patent-phrase-fold4/'
    
    learning_rate = 2e-5
    weight_decay = 0.01
    num_fold = 5
    epochs = 5
    batch_size = 64

In [20]:
test_df = pd.read_csv(f"{CFG.input_path}test.csv")
titles = pd.read_csv('../input/cpc-codes/titles.csv')
test_df.reset_index(inplace=True)
test_df = test_df.merge(titles, left_on='context', right_on='code')
test_df.sort_values(by='index', inplace=True)
test_df.reset_index(inplace=True)
test_df.drop(columns='index', inplace=True)
test_df['input'] = test_df['title']+' '+test_df['anchor']
test_df = test_df.drop(columns=["context", "code", "class", "subclass", "group", "main_group", "anchor", "title", "section"])

In [21]:
tokenizer = AutoTokenizer.from_pretrained(f'{CFG.model_path}uspppm_SKF_0')

In [22]:
class InferDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['input'].values.astype(str)
        self.targets = df['target'].values.astype(str)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = self.inputs[item]
        targets = self.targets[item]
        
        return {
        **tokenizer( inputs, targets )
    }

In [23]:
predictions = []

for fold in range(CFG.num_fold):
    te_dataset = InferDataset(test_df)
    model = AutoModelForSequenceClassification.from_pretrained(f'{CFG.model_path}uspppm_SKF_{fold}', num_labels=1)
    trainer = Trainer(
            model,
            tokenizer=tokenizer
        )
    outputs = trainer.predict(te_dataset)
    prediction = outputs.predictions.reshape(-1)
    predictions.append(prediction)
    del model, prediction; gc.collect()
    torch.cuda.empty_cache() 
pred2 = np.mean(predictions, axis=0)

***** Running Prediction *****
  Num examples = 36
  Batch size = 8


loading configuration file ../input/patent-phrase-fold4/uspppm_SKF_1/config.json
Model config BertConfig {
  "_name_or_path": "../input/patent-phrase-fold4/uspppm_SKF_1",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 39859
}

loading weights file ../input/patent-phrase-fold4/uspppm_SKF_1/pytorch_model.bin
All model checkpoint weights were u

loading configuration file ../input/patent-phrase-fold4/uspppm_SKF_2/config.json
Model config BertConfig {
  "_name_or_path": "../input/patent-phrase-fold4/uspppm_SKF_2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 39859
}

loading weights file ../input/patent-phrase-fold4/uspppm_SKF_2/pytorch_model.bin
All model checkpoint weights were u

loading configuration file ../input/patent-phrase-fold4/uspppm_SKF_3/config.json
Model config BertConfig {
  "_name_or_path": "../input/patent-phrase-fold4/uspppm_SKF_3",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 39859
}

loading weights file ../input/patent-phrase-fold4/uspppm_SKF_3/pytorch_model.bin
All model checkpoint weights were u

loading configuration file ../input/patent-phrase-fold4/uspppm_SKF_4/config.json
Model config BertConfig {
  "_name_or_path": "../input/patent-phrase-fold4/uspppm_SKF_4",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 39859
}

loading weights file ../input/patent-phrase-fold4/uspppm_SKF_4/pytorch_model.bin
All model checkpoint weights were u

In [24]:
MMscaler = MinMaxScaler()
pred2_mm = MMscaler.fit_transform(pred2.reshape(-1,1)).reshape(-1)

In [25]:
submission_2 = pd.DataFrame({
    'id': test_df['id'],
    'score': pred2_mm,
})

submission_2.head(14)

,id,score
0,4112d61851461f60,0.458402
1,09e418c93a776564,0.605043
2,36baf228038e314b,0.436031
3,1f37ead645e7f0c8,0.238926
4,71a5b6ad068d531f,0.000457
5,474c874d0c07bd21,0.476554
6,442c114ed5c4e3c9,0.381174
7,b8ae62ea5e1d8bdb,0.002258
8,faaddaf8fcba8a3f,0.251393
9,ae0262c02566d2ce,1.000000


# 3.anferico/bert-for-patents

In [26]:
import sys
sys.path.append("../input/tez-lib/")

In [27]:
import torch.nn as nn

from scipy import stats
from tez import Tez, TezConfig
from tez.callbacks import EarlyStopping
from transformers import AutoModel, AutoConfig, AutoTokenizer, get_linear_schedule_with_warmup

In [28]:
class args:
    model = "../input/anferico-bert-for-patents/"
    max_len = 32
    accumulation_steps = 1
    batch_size = 64
    epochs = 5
    learning_rate = 2e-5

In [29]:
class PhraseDataset:
    def __init__(self, anchor, target, context, tokenizer, max_len):
        self.anchor = anchor
        self.target = target
        self.context = context
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.anchor)

    def __getitem__(self, item):
        anchor = self.anchor[item]
        context = self.context[item]
        target = self.target[item]

        encoded_text = self.tokenizer.encode_plus(
            context + " " + anchor,
            target,
            padding="max_length",
            max_length=self.max_len,
            truncation=True,
        )
        input_ids = encoded_text["input_ids"]
        attention_mask = encoded_text["attention_mask"]
        token_type_ids = encoded_text["token_type_ids"]

        return {
            "ids": torch.tensor(input_ids, dtype=torch.long),
            "mask": torch.tensor(attention_mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        }

In [30]:
class PhraseModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.model_name = model_name

        config = AutoConfig.from_pretrained(model_name)
        config.update(
            {
                "output_hidden_states": True,
                "add_pooling_layer": True,
                "num_labels": 1,
            }
        )
        self.transformer = AutoModel.from_pretrained(model_name, config=config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.output = nn.Linear(config.hidden_size, 1)

    def forward(self, ids, mask, token_type_ids):
        transformer_out = self.transformer(ids, mask, token_type_ids)
        output = transformer_out.pooler_output
        output = self.dropout(output)
        output = self.output(output)
        return output, 0, {}

In [31]:
df = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/test.csv")

context_mapping = {
    "A": "Human Necessities",
    "B": "Operations and Transport",
    "C": "Chemistry and Metallurgy",
    "D": "Textiles",
    "E": "Fixed Constructions",
    "F": "Mechanical Engineering",
    "G": "Physics",
    "H": "Electricity",
    "Y": "Emerging Cross-Sectional Technologies",
}

df.context = df.context.apply(lambda x: context_mapping[x[0]])

tokenizer = AutoTokenizer.from_pretrained(args.model)
test_dataset = PhraseDataset(
    anchor=df.anchor.values,
    target=df.target.values,
    context=df.context.values,
    tokenizer=tokenizer,
    max_len=args.max_len,
)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file ../input/anferico-bert-for-patents/config.json
Model config BertConfig {
  "_name_or_path": "../input/anferico-bert-for-patents/",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 39859
}

Didn't find file ../input/anferico-bert-for-patents/tokenizer.json. We won't load it.
Didn't find file ../input/anferico-bert-for-patents/added_tokens.json. We won't load it.
Didn't find f

In [32]:
predictions = []

for model_index in range(5):
    model = PhraseModel(model_name=args.model)
    model = Tez(model)
    model_path = f"../input/usppmtez5folds0526/model_f{model_index}.bin"
    config = TezConfig(
        test_batch_size=64,
        device="cuda",
    )
    model.load(model_path, weights_only=True, config=config)

    preds_iter = model.predict(test_dataset)
    final_preds = []
    for preds in preds_iter:
#        preds[preds < 0] = 0
#        preds[preds > 1] = 1
        final_preds.extend(preds.ravel().tolist())
    predictions.append(np.array(final_preds))
    
    del model
    gc.collect()
    torch.cuda.empty_cache() 
    
pred3 = np.mean(predictions, axis=0)

loading configuration file ../input/anferico-bert-for-patents/config.json
Model config BertConfig {
  "_name_or_path": "../input/anferico-bert-for-patents/",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 39859
}

loading weights file ../input/anferico-bert-for-patents/pytorch_model.bin
Some weights of the model checkpoint at ../input/anferico-bert-for-patents/ were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transfo

In [33]:
MMscaler = MinMaxScaler()
pred3_mm = MMscaler.fit_transform(pred3.reshape(-1,1)).reshape(-1)

In [34]:
submission_3 = pd.DataFrame({
    'id': df['id'],
    'score': pred3_mm,
})

submission_3.head(14)

,id,score
0,4112d61851461f60,0.475254
1,09e418c93a776564,0.608357
2,36baf228038e314b,0.398566
3,1f37ead645e7f0c8,0.235248
4,71a5b6ad068d531f,0.000000
5,474c874d0c07bd21,0.461583
6,442c114ed5c4e3c9,0.334973
7,b8ae62ea5e1d8bdb,0.000103
8,faaddaf8fcba8a3f,0.214788
9,ae0262c02566d2ce,1.000000


# 4.Roberta-large

In [35]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(seed=2019)

In [36]:
@dataclass(frozen=True)
class CFG:
    num_workers: Optional[int] = 4
    config_path: Optional[str] = '../input/robertalarge'
    model_path: Optional[str] = '../input/220606-uspppm-roberta7'
    model_name: Optional[str] = 'roberta-large'
    batch_size: Optional[int] = 32
    max_len: Optional[int] = 128
    seed: Optional[int] = 2019
    num_targets: Optional[int] = 1
    n_folds: Optional[int] = 4
    tokenizer = AutoTokenizer.from_pretrained('../input/robertalarge')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file ../input/robertalarge/config.json
Model config RobertaConfig {
  "_name_or_path": "../input/robertalarge",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

Didn't find file ../input/robertalarge/tokenizer.json. We won't load it.
Didn't find file ../input/robertalarge/added_tokens.json. We won't load it.
Didn't find fil

In [37]:
PATH = '../input/us-patent-phrase-to-phrase-matching'
test = pd.read_csv(os.path.join(PATH, 'test.csv'))

In [38]:
context_mapping = {
        "A": "Human Necessities",
        "B": "Operations and Transport",
        "C": "Chemistry and Metallurgy",
        "D": "Textiles",
        "E": "Fixed Constructions",
        "F": "Mechanical Engineering",
        "G": "Physics",
        "H": "Electricity",
        "Y": "Emerging Cross-Sectional Technologies",
}
    
test.context = test.context.apply(lambda x: context_mapping[x[0]])

In [39]:
class PhraseDataset:
    def __init__(self, anchor, target, context, tokenizer, max_len):
        self.anchor = anchor
        self.target = target
        self.context = context
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.anchor)

    def __getitem__(self, item):
        anchor = self.anchor[item]
        context = self.context[item]
        target = self.target[item]

        encoded_text = CFG.tokenizer.encode_plus(
            context + " " + anchor,
            target,
            padding="max_length",
            max_length=self.max_len,
            truncation=True,
        )
        input_ids = encoded_text["input_ids"]
        attention_mask = encoded_text["attention_mask"]

        return {
            "ids": torch.tensor(input_ids, dtype=torch.long),
            "mask": torch.tensor(attention_mask, dtype=torch.long),
        }


In [40]:
def inference_fn(model, test_loader):  
    model.eval()
    predictions = []
    tk0 = tqdm(test_loader, total=len(test_loader))
    for data in tk0:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        with torch.no_grad():
            output = model(ids, mask)
        predictions.append(output.sigmoid().detach().cpu().numpy())
        
    return np.concatenate(predictions)

In [41]:
class PatentModel(torch.nn.Module):
    def __init__(self):
        super(PatentModel, self).__init__()
        hidden_dropout_prob: float = 0.1
        layer_norm_eps: float = 1e-7

        config = AutoConfig.from_pretrained(CFG.config_path)

        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
            }
        )
        
        self.transformer = AutoModel.from_pretrained(CFG.config_path, config=config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        self.output = nn.Linear(config.hidden_size, CFG.num_targets)
        
    def forward(self, ids, mask):
        transformer_out = self.transformer(input_ids=ids, attention_mask=mask)
        last_hidden_states = transformer_out[0]
        last_hidden_states = self.dropout(torch.mean(last_hidden_states, 1))
        logits1 = self.output(self.dropout1(last_hidden_states))
        logits2 = self.output(self.dropout2(last_hidden_states))
        logits3 = self.output(self.dropout3(last_hidden_states))
        logits4 = self.output(self.dropout4(last_hidden_states))
        logits5 = self.output(self.dropout5(last_hidden_states))
        logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
        
        return logits

In [42]:
def run_fold(test, fold, seed=42):    
    
    seed_everything(seed)
    
    test_dataset = PhraseDataset(
        test.anchor.values,
        test.target.values,
        test.context.values,
        CFG.tokenizer, 
        CFG.max_len
    ) 
    
    test_loader = DataLoader(test_dataset, 
                              batch_size=CFG.batch_size * 2, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    model = PatentModel()
    
    model.load_state_dict(
        torch.load(f'{CFG.model_path}/{CFG.model_name.replace("-","_")}_patent_model_{fold}.pth',
        map_location=torch.device('cuda')
        )
    )
    
    model.to(device)

    preds = inference_fn(model, test_loader)
    
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
    return preds

In [43]:
def inference_model(test, seed):
    
    predictions = []
    
    for f in range(CFG.n_folds):    
        preds = run_fold(test, f, seed) 
        predictions.append(preds)
        
    test_preds = np.column_stack(predictions)
        
    return test_preds

In [44]:
if __name__ == '__main__':
    pred4 =  np.mean(inference_model(test, CFG.seed),axis=1)

loading configuration file ../input/robertalarge/config.json
Model config RobertaConfig {
  "_name_or_path": "../input/robertalarge",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file ../input/robertalarge/pytorch_model.bin
Some weights of the model checkpoint at ../input/robertalarge were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.d

In [45]:
MMscaler = MinMaxScaler()
pred4_mm = MMscaler.fit_transform(pred4.reshape(-1,1)).reshape(-1)

In [46]:
submission_4 = pd.DataFrame({
    'id': test['id'],
    'score': pred4_mm,
})

submission_4.head(14)

,id,score
0,4112d61851461f60,0.560307
1,09e418c93a776564,0.757374
2,36baf228038e314b,0.445688
3,1f37ead645e7f0c8,0.309286
4,71a5b6ad068d531f,0.064136
5,474c874d0c07bd21,0.481080
6,442c114ed5c4e3c9,0.446558
7,b8ae62ea5e1d8bdb,0.000065
8,faaddaf8fcba8a3f,0.361119
9,ae0262c02566d2ce,1.000000


# 5.deberta_v3

In [47]:
class CFG_DEB_SIMPLE:
    input_path = '../input/us-patent-phrase-to-phrase-matching/'
    model_path = '../input/deberta-v3-large/deberta-v3-large'
    batch_size = 24
    num_workers = 2
    num_fold = 4
    max_input_length = 130

In [48]:
class TestDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_length):
        self.text = df['text'].values.astype(str)
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        
    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        inputs = self.text[item]
        
        inputs = self.tokenizer(inputs,
                    max_length=self.max_input_length,
                    padding='max_length',
                    truncation=True)
        
        return torch.as_tensor(inputs['input_ids'], dtype=torch.long), \
               torch.as_tensor(inputs['token_type_ids'], dtype=torch.long), \
               torch.as_tensor(inputs['attention_mask'], dtype=torch.long)
    
    
class Custom_Bert_Simple(nn.Module):
    def __init__(self, model_path):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_path)
        config.num_labels = 1
        self.base = AutoModelForSequenceClassification.from_config(config=config)
        dim = config.hidden_size
        self.dropout = nn.Dropout(p=0)
        self.cls = nn.Linear(dim,1)
        
    def forward(self, input_ids, attention_mask, token_type_ids, labels=None):
        base_output = self.base(input_ids=input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids
        )

        return base_output[0]

In [49]:
def valid_fn(valid_loader, model, device):
    model.eval()
    preds = []
    labels = []
    
    for step, batch in enumerate(valid_loader):
        input_ids, token_type_ids, attention_mask = [i.to(device) for i in batch]
    
        with torch.no_grad():
            y_preds = model(input_ids, attention_mask, token_type_ids)
        
        preds.append(y_preds.to('cpu').numpy())
    
    predictions = np.concatenate(preds)
    
    return predictions


min_max_scaler = MinMaxScaler()

def upd_outputs(data, is_trim=True, is_minmax=True, is_reshape=True):
    """\o/"""
    if is_trim == True:
        data = np.where(data <=0, 0, data)
        data = np.where(data >=1, 1, data)

    if is_minmax ==True:
        data = min_max_scaler.fit_transform(data)
    
    if is_reshape == True:
        data = data.reshape(-1)
        
    return data


def _upd_score_between(data, thresholds, value):
    """\o/"""
    mask_th = data.between(*thresholds, inclusive='both')
    data[mask_th] = value


def upd_score(data, th_dict=None):
    """\o/"""
    if isinstance(data, pd.Series):
        result = data.copy()
    else:
        return data

    if not th_dict:        
        th_dict = {
            '0': 0.02,
            '.25': (0.24, 0.26),
            '.50': (0.49, 0.51),
            '.75': (0.74, 0.76),
            '1': 0.98
        }

    if isinstance(th_dict, dict):    
        th0 = th_dict.get('0')
        th25 = th_dict.get('.25')
        th50 = th_dict.get('.50')
        th75 = th_dict.get('.75')
        th100 = th_dict.get('1')
    else:
        return data
    
    if th0:
        if isinstance(th0, float):
            th0 = (result.min(), th0)
        
        if isinstance(th0, tuple):
            _upd_score_between(result, th0, 0)
    
    if th25 and isinstance(th25, tuple):
        _upd_score_between(result, th25, 0.25)

    if th50 and isinstance(th50, tuple):
        _upd_score_between(result, th50, 0.50)
            
    if th75 and isinstance(th75, tuple):
        _upd_score_between(result, th75, 0.75)
            
    if th100:
        if isinstance(th100, float):
            th100 = (th100, result.max())
        
        if isinstance(th100, tuple):
            _upd_score_between(result, th100, 1)

    return result

In [50]:
test_df = pd.read_csv(f"{CFG_DEB_SIMPLE.input_path}test.csv")

# ====================================================
# CPC Data
# ====================================================
cpc_texts = torch.load("../input/folds-dump-the-two-paths-fix/cpc_texts.pth")
test_df['context_text'] = test_df['context'].map(cpc_texts)
test_df['text'] = test_df['anchor'] + '[SEP]' + test_df['target'] + '[SEP]'  + test_df['context_text']
test_df['text'] = test_df['text'].apply(str.lower)
display(test_df.head())

,id,anchor,target,context,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,PHYSICS. OPTICS,opc drum[sep]inorganic photoconductor drum[sep...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow[sep]altering gas flow[sep]mech...
2,36baf228038e314b,lower trunnion,lower locating,B60,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion[sep]lower locating[sep]performi...
3,1f37ead645e7f0c8,cap component,upper portion,D06,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component[sep]upper portion[sep]textiles; ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation[sep]artificial neural netwo...


In [51]:
tokenizer_deberta_v3 = AutoTokenizer.from_pretrained(CFG_DEB_SIMPLE.model_path)

loading configuration file ../input/deberta-v3-large/deberta-v3-large/config.json
Model config DebertaV2Config {
  "_name_or_path": "../input/deberta-v3-large/deberta-v3-large",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.16.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

Didn't find file ../input/deberta-v3-large/deberta-v3-large/added_tokens.json. We wo

In [52]:
predictions = []

te_dataset = TestDataset(test_df, tokenizer_deberta_v3, CFG_DEB_SIMPLE.max_input_length)
te_dataloader = DataLoader(te_dataset,
                              batch_size=CFG_DEB_SIMPLE.batch_size, shuffle=False,
                              num_workers=CFG_DEB_SIMPLE.num_workers,
                              pin_memory=True, drop_last=False)

deberta_simple_path = "../input/us-patent-deberta-simple/microsoft_deberta-v3-large"

for fold in tqdm(range(CFG_DEB_SIMPLE.num_fold)):
    fold_path = f"{deberta_simple_path}_best{fold}.pth"
    
    model = Custom_Bert_Simple(CFG_DEB_SIMPLE.model_path)
    model.load_state_dict(torch.load(fold_path)['model'])
    model.to('cuda')
    
    prediction = valid_fn(te_dataloader, model, 'cuda')
    predictions.append(prediction)
    
    del model, prediction; gc.collect()
    torch.cuda.empty_cache() 

  0%|          | 0/4 [00:00<?, ?it/s]loading configuration file ../input/deberta-v3-large/deberta-v3-large/config.json
Model config DebertaV2Config {
  "_name_or_path": "../input/deberta-v3-large/deberta-v3-large",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.16.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

 25%|██▌       | 1/4 [00:29<01:28, 29.38s/it]lo

In [53]:
print("folds:", len(predictions))
print("rows: ", len(predictions[0]))
print("score:", predictions[0][0])

folds: 4
rows:  36
score: [0.5036294]


In [54]:
n_predictions = 14

In [55]:
# first fold
predictions[0][:n_predictions]

array([[ 0.5036294 ],
       [ 0.4791839 ],
       [ 0.49305084],
       [ 0.2301636 ],
       [ 0.4067986 ],
       [ 0.5016197 ],
       [ 0.516191  ],
       [-0.02492346],
       [ 0.1932471 ],
       [ 0.93988395],
       [ 0.2218402 ],
       [ 0.27013338],
       [ 0.7560612 ],
       [ 0.832283  ]], dtype=float32)

In [56]:
# np.where(x<=0, 0, x) .. >> min_max.fit_transform(x) >> x.reshape(-1)
upd_predictions = [upd_outputs(x, is_trim=False) for x in predictions]

In [57]:
# === add np.median ===
add_preds = []
for x in zip(*upd_predictions):
    add_preds.append(np.median(x, axis=0))
    
upd_predictions.append(add_preds)

In [58]:
# === add np.mean ===
add_preds = []
for x in zip(*upd_predictions):
    add_preds.append(np.mean(x, axis=0))
    
upd_predictions.append(add_preds)

In [59]:
pred5 = np.mean(upd_predictions, axis=0)

In [60]:
print(*pred5[:n_predictions])

0.31270644 0.59676236 0.5130505 0.28154138 0.09131227 0.4976574 0.46910086 0.0035888886 0.25367817 0.9989567 0.25185874 0.2716439 0.77297974 0.8849662


In [61]:
submission_5 = pd.DataFrame({
    'id': test_df['id'],
    'score': pred5,
})

submission_5.head(14)

,id,score
0,4112d61851461f60,0.312706
1,09e418c93a776564,0.596762
2,36baf228038e314b,0.513050
3,1f37ead645e7f0c8,0.281541
4,71a5b6ad068d531f,0.091312
5,474c874d0c07bd21,0.497657
6,442c114ed5c4e3c9,0.469101
7,b8ae62ea5e1d8bdb,0.003589
8,faaddaf8fcba8a3f,0.253678
9,ae0262c02566d2ce,0.998957


# 6.electra

In [62]:
# ====================================================
# Directory settings
# ====================================================
import os

INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [63]:
class CFG:
    num_workers=4
    path="../input/electrav1/"
    config_path=path+'config.pth'
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=133
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    tokenizer = AutoTokenizer.from_pretrained('../input/electrav1/tokenizer/tokenizer/')

Didn't find file ../input/electrav1/tokenizer/tokenizer/added_tokens.json. We won't load it.
loading file ../input/electrav1/tokenizer/tokenizer/vocab.txt
loading file ../input/electrav1/tokenizer/tokenizer/tokenizer.json
loading file None
loading file ../input/electrav1/tokenizer/tokenizer/special_tokens_map.json
loading file ../input/electrav1/tokenizer/tokenizer/tokenizer_config.json


In [64]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [65]:
# ====================================================
# Data Loading
# ====================================================
test = pd.read_csv(INPUT_DIR+'test.csv')
print(f"test.shape: {test.shape}")
display(test.head())

test.shape: (36, 4)


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [66]:
# ====================================================
# CPC Data
# ====================================================
cpc_texts = torch.load("../input/pppm-deberta-v3-large-baseline-w-w-b-train/cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())

,id,anchor,target,context,context_text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,PHYSICS. OPTICS
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...
2,36baf228038e314b,lower trunnion,lower locating,B60,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...
3,1f37ead645e7f0c8,cap component,upper portion,D06,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE


In [67]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

,id,anchor,target,context,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,PHYSICS. OPTICS,opc drum[SEP]inorganic photoconductor drum[SEP...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow[SEP]altering gas flow[SEP]MECH...
2,36baf228038e314b,lower trunnion,lower locating,B60,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion[SEP]lower locating[SEP]PERFORMI...
3,1f37ead645e7f0c8,cap component,upper portion,D06,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component[SEP]upper portion[SEP]TEXTILES; ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation[SEP]artificial neural netwo...


In [68]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

In [69]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self.layer_norm1 = nn.LayerNorm(self.config.hidden_size)
        self._init_weights(self.attention)
        self.linear = nn.Linear(self.config.hidden_size, 1)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_state = outputs[0]
        input_mask_expanded = inputs["attention_mask"].unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        out = sum_embeddings / sum_mask
        
        out = self.layer_norm1(out)
        output = self.fc(out)
        
        
        return output

In [70]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [71]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in range(4):
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(f'../input/electrav1/google-electra-large-discriminator_fold{fold}_best/google-electra-large-discriminator_fold{fold}_best.pth',
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
pred7 = np.mean(predictions, axis=0)

100%|██████████| 2/2 [00:01<00:00,  1.58it/s]


In [72]:
MMscaler = MinMaxScaler()
pred7_mm = MMscaler.fit_transform(pred7.reshape(-1,1)).reshape(-1)

In [73]:
submission_7 = pd.DataFrame({
    'id': test['id'],
    'score': pred7_mm,
})

submission_7.head(14)

,id,score
0,4112d61851461f60,0.609093
1,09e418c93a776564,0.726084
2,36baf228038e314b,0.326839
3,1f37ead645e7f0c8,0.323469
4,71a5b6ad068d531f,0.000843
5,474c874d0c07bd21,0.522739
6,442c114ed5c4e3c9,0.379698
7,b8ae62ea5e1d8bdb,0.000000
8,faaddaf8fcba8a3f,0.333683
9,ae0262c02566d2ce,1.000000


In [74]:
class CFG:
    num_workers=4
    path="../input/electrav5/"
    config_path=path+'config.pth'
    batch_size=32
    fc_dropout=0.1
    target_size=1
    max_len=133
    seed=34
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    tokenizer = AutoTokenizer.from_pretrained('../input/google-electra-large-discriminator')

Didn't find file ../input/google-electra-large-discriminator/added_tokens.json. We won't load it.
loading file ../input/google-electra-large-discriminator/vocab.txt
loading file ../input/google-electra-large-discriminator/tokenizer.json
loading file None
loading file ../input/google-electra-large-discriminator/special_tokens_map.json
loading file ../input/google-electra-large-discriminator/tokenizer_config.json


In [75]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in range(4):
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(f'../input/electrav5/google-electra-large-discriminator_fold{fold}_best.pth',
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
pred8 = np.mean(predictions, axis=0)

100%|██████████| 2/2 [00:01<00:00,  1.40it/s]


In [76]:
MMscaler = MinMaxScaler()
pred8_mm = MMscaler.fit_transform(pred8.reshape(-1,1)).reshape(-1)

In [77]:
submission_8 = pd.DataFrame({
    'id': test['id'],
    'score': pred8_mm,
})

submission_8.head(14)

,id,score
0,4112d61851461f60,0.599164
1,09e418c93a776564,0.794765
2,36baf228038e314b,0.342252
3,1f37ead645e7f0c8,0.334473
4,71a5b6ad068d531f,0.072055
5,474c874d0c07bd21,0.543924
6,442c114ed5c4e3c9,0.340275
7,b8ae62ea5e1d8bdb,0.000024
8,faaddaf8fcba8a3f,0.367220
9,ae0262c02566d2ce,1.000000


# 7.xlm roberta large

In [78]:
class CFG:
    input_path = '../input/us-patent-phrase-to-phrase-matching/'
    model_path = '../input/xlm-roberta-large-5folds/',
    model_num = 1
    num_fold = 5

In [79]:
titles = pd.read_csv('../input/cpc-codes/titles.csv')
test = pd.read_csv(f"{CFG.input_path}test.csv")
test.reset_index(inplace=True)
test = test.merge(titles, left_on='context', right_on='code')
test.sort_values(by='index', inplace=True)
test.drop(columns='index', inplace=True)
test.reset_index(inplace=True)
test['input'] = test['title']+'[SEP]'+test['anchor']
test = test.drop(columns=["context", "code", "class", "subclass", "group", "main_group", "anchor", "title", "section"])

In [80]:
class InferDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['input'].values.astype(str)
        self.targets = df['target'].values.astype(str)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = self.inputs[item]
        targets = self.targets[item]
        
        return {
        **tokenizer( inputs, targets )
    }

In [81]:
predictions = []
MMscaler = MinMaxScaler()

for i in range (CFG.model_num):   
    tokenizer = AutoTokenizer.from_pretrained(f'{CFG.model_path[i]}fold0')

    def process_test(unit):
            return {
            **tokenizer( unit['input'], unit['target'])
        }
    
    def process_valid(unit):
        return {
        **tokenizer( unit['input'], unit['target']),
        'label': unit['score']
    }
    
    test_ds = InferDataset(test)
    
    for fold in range(CFG.num_fold):        
        model = AutoModelForSequenceClassification.from_pretrained(f'{CFG.model_path[i]}fold{fold}', 
                                                                   num_labels=1)
        trainer = Trainer(
                model,
                tokenizer=tokenizer,
            )
        
        outputs = trainer.predict(test_ds)
        prediction = MMscaler.fit_transform(outputs.predictions.reshape(-1, 1)).reshape(-1)
        predictions.append(prediction)
        
        del model, prediction; gc.collect()
        torch.cuda.empty_cache()
        
pred9_mm = np.mean(predictions, axis=0)
# np.clip(predictions, 0.0, 1.0, out=predictions)

Didn't find file ../input/xlm-roberta-large-5folds/fold0/added_tokens.json. We won't load it.
loading file ../input/xlm-roberta-large-5folds/fold0/sentencepiece.bpe.model
loading file ../input/xlm-roberta-large-5folds/fold0/tokenizer.json
loading file None
loading file ../input/xlm-roberta-large-5folds/fold0/special_tokens_map.json
loading file ../input/xlm-roberta-large-5folds/fold0/tokenizer_config.json
loading configuration file ../input/xlm-roberta-large-5folds/fold0/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "../input/xlm-roberta-large-5folds/fold0",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,

loading configuration file ../input/xlm-roberta-large-5folds/fold1/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "../input/xlm-roberta-large-5folds/fold1",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file ../input/xlm-r

loading configuration file ../input/xlm-roberta-large-5folds/fold2/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "../input/xlm-roberta-large-5folds/fold2",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file ../input/xlm-r

loading configuration file ../input/xlm-roberta-large-5folds/fold3/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "../input/xlm-roberta-large-5folds/fold3",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file ../input/xlm-r

loading configuration file ../input/xlm-roberta-large-5folds/fold4/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "../input/xlm-roberta-large-5folds/fold4",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file ../input/xlm-r

In [82]:
submission_9 = pd.DataFrame({
    'id': test['id'],
    'score': pred9_mm,
})

# post-process

In [83]:
thresholds_dict = {
    '0': 0.02,
    '.25': (0.24, 0.26),
    '.50': (0.49, 0.51),
    '.75': (0.74, 0.76),
    '1': 0.98
}

# No.1
submission_1['score'] = upd_score(submission_1['score'], thresholds_dict)
# No.2
submission_2['score'] = upd_score(submission_2['score'], thresholds_dict)
# No.3
submission_3['score'] = upd_score(submission_3['score'], thresholds_dict)
# No.4
submission_4['score'] = upd_score(submission_4['score'], thresholds_dict)
# No.5
submission_5['score'] = upd_score(submission_5['score'], thresholds_dict)
# No.6
submission_6['score'] = upd_score(submission_6['score'], thresholds_dict)
# No.7
submission_7['score'] = upd_score(submission_7['score'], thresholds_dict)
# No.8
submission_8['score'] = upd_score(submission_8['score'], thresholds_dict)
# No.9
submission_9['score'] = upd_score(submission_9['score'], thresholds_dict)

# Ensemble

In [84]:
w1 = 0.14
w2 = 0.07
w3 = 0.11
w4 = 0.05
w5 = 0.28
w6 = 0.10
w7 = 0.12
w8 = 0.08
w9 = 0.05

In [85]:
submission_1['score'] = submission_1['score'] * w1 + submission_2['score'] * w2 + submission_3['score'] * w3 + submission_4['score'] * w4 + submission_5['score'] * w5 + submission_6['score'] * w6 + submission_7['score'] * w7 + submission_8['score'] * w8 + submission_9['score'] * w9

# Submission

In [86]:
submission_1.head()

,id,score
0,4112d61851461f60,0.462946
1,09e418c93a776564,0.674210
2,36baf228038e314b,0.436187
3,1f37ead645e7f0c8,0.278490
4,71a5b6ad068d531f,0.037007


In [87]:
submission_1.to_csv('submission.csv', index=False)